In [ ]:
#Required Libraries 

#Miscallenous 
import os
import json
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pydicom
from tqdm import tqdm

#Deep learning Libraries 
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from keras.initializers import Constant
from keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.python.ops import array_ops
from keras import backend as K
import tensorflow as tf
import keras
from keras.applications import Xception
from keras.models import Model, load_model
from keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D

#Evaluation 
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import log_loss

# Let's first initiate the paths to load the data 

In [ ]:
BASE_PATH = #Your base file, contains all the data. 
TRAIN_DIR = 'stage_2_train/'
TEST_DIR = 'stage_2_test/'
train_df = pd.read_csv(BASE_PATH + 'stage_2_train.csv')

In [ ]:
train_df

Then, split the name and type sections of each ID 

In [ ]:
x = pd.read_csv(BASE_PATH + 'stage_2_sample_submission.csv')

train_df['filename'] = train_df['ID'].apply(lambda x: "ID_" + x.split('_')[1] + ".png")
train_df['type'] = train_df['ID'].apply(lambda x: x.split('_')[2])
x['filename'] = x['ID'].apply(lambda x: "ID_" + x.split('_')[1] + ".png")
x['type'] = x['ID'].apply(lambda x: x.split('_')[2])

print(train_df.shape)
train_df.head()

In [ ]:
test_df = pd.DataFrame(x.filename.unique(), columns=['filename'])
print(test_df.shape)
test_df.head()

In [ ]:
test_df = x[['Label', 'filename', 'type']].drop_duplicates().pivot(
    index='filename', columns='type', values='Label').reset_index()
print(test_df.shape)
test_df

Next, we pivot the dataframe to create the one that will be use to load the data in the datagenerators 

In [ ]:
#Select 400,000 random images 

np.random.seed(2019)
sample_files = np.random.choice(os.listdir(BASE_PATH + TRAIN_DIR), 400000)
sample_df = train_df[train_df.filename.apply(lambda x: x.replace('.png', '.dcm')).isin(sample_files)]

In [ ]:
pivot_df = sample_df[['Label', 'filename', 'type']].drop_duplicates().pivot(
    index='filename', columns='type', values='Label').reset_index()
print(pivot_df.shape)
pivot_df

Then, we create the validation dataframe, taking a 15% split 

In [ ]:
#For the balanced data approach: do not run the randomizer, pivot from the train_df
#then, uncomment the following cells: 

In [ ]:
#This speoarates the training data into two data frame, one only containing the 
#images ID labeled as "No hemorrages" and the ones labeled as "any hemorrhage"

#For the "only subudral" approach: the same can be done, but select 'subdural' instead of 'any'
#Additionaly, the Output layer of the model needs to be changed to return only 1 probabilities and not 6
#and the DataGenerators need to be modified to only consider the 'subdural' labels


#one_df = pivot_df.drop(pivot_df.loc[pivot_df['any']==0].index)
#one_df

#zero_df = pivot_df.drop(pivot_df.loc[pivot_df['any']==1].index)
#zero_df


In [ ]:
#pivot_df = pd.concat([zero_df, one_df])
#pivot_df

In [ ]:
validation_df = pivot_df.sample(int(len(pivot_df) * 0.15))  
validation_df

In [ ]:
training_df = pivot_df[~(pivot_df.filename.isin(validation_df.filename))]
training_df

In [ ]:
#Take a sample of 20000 from the testing set, just to make predicitions slightly faster 

np.random.seed(2019)
sample_test = np.random.choice(os.listdir(BASE_PATH + TEST_DIR), 20000) 
test_sample_df = test_df[test_df.filename.apply(lambda x: x.replace('.png', '.dcm')).isin(sample_test)]

In [ ]:
test_sample_df

In [ ]:
#We initiatie a list of labels, for future validation (it will be required when computing the ROC
#curves, the confusion matrices, etc)
y_true = []
for i in range(len(validation_df)): 
    y_true.append(validation_df.iloc[i,1])
        

In [ ]:
len(y_true)

In [ ]:
#This cell is necessary for the balanaced data approach, to create the list of the 
#IDs included in the training dataframe used in this approach 

#pivot_df['filename'] = pivot_df.filename.apply(lambda x: x.replace('.png', '.dcm'))
#sample_files = sample_df["filename"].to_numpy()
#len(sample_files)

In [ ]:
print(training_df.head())
print(validation_df.head())

# The next section will include the image preprocessing functions 

In [ ]:
def get_pixels_hu(scan): 
    image = np.stack([scan.pixel_array])
    image = image.astype(np.int16) 
    
    image[image == -2000] = 0
    
    intercept = scan.RescaleIntercept
    slope = scan.RescaleSlope
    
    if slope != 1: 
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
    
    image += np.int16(intercept) 
    
    return np.array(image, dtype=np.int16)

In [ ]:
#for the windowing function, all credit goes to:
#https://github.com/darraghdog/rsna/blob/master/scripts/prepare_meta_dicom.py

def apply_window(image, center, width):
    image = image.copy()
    min_value = center - width // 2
    max_value = center + width // 2
    image[image < min_value] = min_value
    image[image > max_value] = max_value
    return image


def apply_window_policy(image):

    image1 = apply_window(image, 40, 80) # brain
    image2 = apply_window(image, 80, 200) # subdural
    image3 = apply_window(image, 40, 380) # bone
    image1 = (image1 - 0) / 80
    image2 = (image2 - (-20)) / 200
    image3 = (image3 - (-150)) / 380
    image = np.array([
        image1 - image1.mean(),
        image2 - image2.mean(),
        image3 - image3.mean(),
    ]).transpose(1,2,0)

    return image

In [ ]:
save_dir = '/save_dir/'   # Save the new images in this file 
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
def save_and_resize(filenames, load_dir):    
    
    for filename in tqdm(filenames):
        try:
            path = load_dir + filename
            new_path = save_dir + filename.replace('.dcm', '.png')
            dcm = pydicom.dcmread(path)
            image = get_pixels_hu(dcm)
            image = apply_window_policy(image[0])
            image -= image.min((0,1))
            image = (255*image).astype(np.uint8)
            image = cv2.resize(image, (299, 299)) 
            final_image = cv2.imwrite(new_path, image)
            
        except ValueError:
            continue

In [ ]:
save_and_resize(filenames=os.listdir(BASE_PATH + TRAIN_DIR), load_dir=BASE_PATH + TRAIN_DIR)
save_and_resize(filenames=os.listdir(BASE_PATH + TEST_DIR), load_dir=BASE_PATH + TEST_DIR)

# Now, we can initiate the Xception model

In [ ]:
def create_model():    
    base_model = Xception(weights = 'imagenet', include_top = False, input_shape = (299,299,3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.15)(x)
    y_pred = Dense(6, activation = 'sigmoid')(x)
    #y_pred = Dense(1, activation = 'sigmoid')(x)  #(only subdural approach)

    return Model(inputs = base_model.input, outputs = y_pred)

In [ ]:
model = create_model()

model.compile(optimizer = Adam(learning_rate = 0.00005), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])
            #metrics = [tf.keras.metrics.AUC()])
            #metrics = [tf.keras.metrics.SensitivityAtSpecificity(0.5)])
            #metrics = [tf.keras.metrics.SpecificityAtSensitivity(0.5)]) for the additional approaches

# And the data generators 

In [ ]:
BATCH_SIZE = 16 # or 32

def create_datagen():
    return ImageDataGenerator()

def create_test_gen():
    return ImageDataGenerator().flow_from_dataframe(
        test_sample_df,
        directory=  '/save_dir/',
        x_col='filename',
        class_mode=None,
        target_size=(299, 299),
        batch_size=BATCH_SIZE,
        shuffle=False
    )

def create_train_gen(datagen):
    return datagen.flow_from_dataframe(
        training_df, 
        directory='/save_dir/',
        
        x_col='filename', 
        y_col=['any', 'epidural', 'intraparenchymal', 
               'intraventricular', 'subarachnoid', 'subdural'], #Remove all but subudral for the only subdrual approach
        class_mode='other',
        target_size=(299, 299),
        batch_size=BATCH_SIZE,   
       
    )
def create_val_gen(datagen): 
    return datagen.flow_from_dataframe(
        validation_df, 
        directory='/save_dir/',
        
        x_col='filename', 
        y_col=['any', 'epidural', 'intraparenchymal', 
               'intraventricular', 'subarachnoid', 'subdural'], #Remove all but subudral for the only subdrual approach
        class_mode='other',
        target_size=(299, 299),
        batch_size=BATCH_SIZE,
        shuffle=False,
    )

In [ ]:
generator = create_datagen()
train_gen = create_train_gen(generator)
val_gen = create_val_gen(generator)
test_gen = create_test_gen()

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

# This cell runs the model

In [ ]:
checkpoint = ModelCheckpoint(
    'effnetb4.h5', 
    monitor='val_loss', 
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False,
    mode='auto'
)
Early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=1, 
                                              mode='auto', baseline=None, restore_best_weights=False)

total_steps = sample_files.shape[0] // BATCH_SIZE
total_steps = total_steps // 4
history = model.fit_generator(
    train_gen,
    steps_per_epoch = total_steps,
    validation_data=val_gen,
    validation_steps=total_steps * 0.15,
    callbacks=[checkpoint, Early_stop],
    epochs=10
)

In [ ]:
acc = history.history['accuracy'] #This needs to be changed if the metric is changed too! 
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'g', label='Validation acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.title('Training and validation accuracy')
plt.legend()
fig = plt.figure()
fig.savefig('acc.png')


plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'g', label='Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and validation loss')

plt.legend()
plt.show()

# Now we make predictions: 
first on the validation set, to ensure our model predicts hemorrhages correct, and then on the test set

In [ ]:
val_preds = model.predict_generator(val_gen, verbose = 1)

In [ ]:
val_preds

In [ ]:
#Initiate a list of rounded predictions. 

y_preds = []
for i in range(len(val_preds)):
    y_preds.append(0)
    for value in val_preds[i]: 
        if value > 0.5: 
            y_preds[i] = 1
            break
            
        
len(y_preds)

In [ ]:
from sklearn.metrics import roc_curve
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_true, y_preds)

In [ ]:
from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
print('2*2 Confusion Matrix')
print(confusion_matrix(y_true, y_preds))
cm = confusion_matrix(y_true, y_preds)

In [ ]:
import itertools   
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm_labels = ['no hemorrhage', 'has hemorrhage']

In [ ]:
plot_confusion_matrix(cm=cm, classes=cm_labels, title='Confusion Matrix')

In [ ]:
predictions_list = []
for pred in val_preds: 
    predictions_list.append(pred)

len(predictions_list)

In [ ]:
validation_frame = validation_df.drop(['filename'], axis=1)
validation_frame 

In [ ]:
if len(predictions_list) == len(validation_frame): 
    validation_frame.iloc[:,:] = predictions_list
else: 
    print("fix this issue")

In [ ]:
validation_frame.insert(0, "filename", validation_df.filename)
validation_frame.insert(7, "true_any" ,validation_df.iloc[:,1])
validation_frame.insert(8, "true_epidural", validation_df.epidural)
validation_frame.insert(9, "true_intraparenchymal", validation_df.intraparenchymal)
validation_frame.insert(10, "true_intraventricular", validation_df.intraventricular)
validation_frame.insert(11, "true_subarachnoid", validation_df.subarachnoid)
validation_frame.insert(12, "true_subdural", validation_df.subdural)

In [ ]:
validation_frame

In [ ]:
for i in range(100): 
    if validation_frame.iloc[i,1] > 0.5: 
        print("ID is : " + str(validation_frame.iloc[i,0]))
        for j in range(1,7): 
            print("predicition = " +  str(validation_frame.iloc[i,j]) )
        for k in range(7,13): 
            print("true predicition = " +  str(validation_frame.iloc[i,k]))


In [ ]:
test_preds = model.predict_generator(test_gen, verbose = 1)

In [ ]:
predictions_list_test = []
for pred in test_preds: 
    predictions_list_test.append(pred)

In [ ]:
test_frame =  test_sample_df.drop(['filename'], axis=1)
test_frame

In [ ]:
test_frame.iloc[:,:] = predictions_list_test
test_frame

In [ ]:
test_frame.insert(0, "filename", test_df.filename)
test_frame

In [ ]:
from PIL import Image
for i in range(20): 
  
    for j in range(1,7): 
        if test_frame.iloc[i,j] > 0.8: 
            path = '/kaggle/tmp/' + str(test_frame.iloc[i,0])
            img = Image.open(path)
            plt.imshow(img)
            print(str(test_frame.iloc[i,0]) + " has a probability: "  + str(test_frame.iloc[i,j]) + " for a '" + str(test_frame.columns[j]) + "' type of hemorrhage")
            plt.show()
          

Lastly, a heatmap of the activation of the last layer of the model is made, taking the image with the highest prediction score in the validation set. This returns a nice methods to see what does the model "sees"

In [ ]:
any_preds = validation_frame['any']
max_index = any_preds.idxmax()
max_index

In [ ]:
def img_to_heatmap(): 
    highest_predicted_img = validation_frame.loc[max_index,'filename']
    if validation_frame.loc[max_index, 'true_any'] == 1:
        return highest_predicted_img

In [ ]:
highest_predicted_img =  img_to_heatmap()
highest_predicted_img

In [ ]:
#heatmap 
#The code used to show the heatmake was taken from: https://keras.io/examples/vision/grad_cam/
#Only slightly modified to fit this workflow and return the image with the highest predicition from the validtion set 
from IPython.display import Image, display

preprocess_input = keras.applications.xception.preprocess_input
decode_predictions = keras.applications.xception.decode_predictions

last_conv_layer_name = "block14_sepconv2_act"

img_path = '/kaggle/tmp/' + highest_predicted_img
display(Image(img_path))

In [ ]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    
    grads = tape.gradient(class_channel, last_conv_layer_output)
    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()


In [ ]:
img_size = (299, 299)

img_array = preprocess_input(get_img_array(img_path, size=img_size))

model.layers[-1].activation = None

heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

# Display heatmap
plt.matshow(heatmap)
plt.show()

In [ ]:
import matplotlib.cm as cm
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    display(Image(cam_path))

In [ ]:
save_and_display_gradcam(img_path, heatmap)